# 问题回答基准测试：Paul Graham Essay（保罗·格雷厄姆Essay）

在这里，我们将介绍如何对Paul Graham Essay上的问题回答任务进行性能基准测试。

强烈建议您在启用跟踪的情况下进行任何评估/基准测试。有关跟踪是什么以及如何设置的说明，请参见[此处]（https://langchain.readthedocs.io/en/latest/tracing.html）。


In [1]:
# Comment this out if you are NOT using tracing
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

## 加载数据
首先，让我们加载数据。

In [2]:
from langchain.evaluation.loading import load_dataset
dataset = load_dataset("question-answering-paul-graham")

Found cached dataset json (/Users/harrisonchase/.cache/huggingface/datasets/LangChainDatasets___json/LangChainDatasets--question-answering-paul-graham-76e8f711e038d742/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

## 设置链
现在，我们需要创建一些用于执行问题回答的流水线。首先是在要问的数据上创建索引。

In [3]:
from langchain.document_loaders import TextLoader
loader = TextLoader("../../modules/paul_graham_essay.txt")

In [4]:
from langchain.indexes import VectorstoreIndexCreator

In [5]:
vectorstore = VectorstoreIndexCreator().from_loaders([loader]).vectorstore

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


现在我们可以创建一个问题回答链。

In [6]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [7]:
chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=vectorstore.as_retriever(), input_key="question")

## 进行预测

首先，我们可以一次预测一个数据点。以这种粒度运行可以让我们详细地探索输出，并且比对多个数据点的运行速度更快。

In [18]:
chain(dataset[0])

{'question': 'What were the two main things the author worked on before college?',
 'answer': 'The two main things the author worked on before college were writing and programming.',
 'result': ' Writing and programming.'}

## 进行多次预测
现在我们可以进行多次预测。

In [9]:
predictions = chain.apply(dataset)

## 评估性能
现在我们可以评估预测结果。首先，我们可以通过肉眼观察它们。

In [10]:
predictions[0]

{'question': 'What were the two main things the author worked on before college?',
 'answer': 'The two main things the author worked on before college were writing and programming.',
 'result': ' Writing and programming.'}

接下来，我们可以使用语言模型对它们进行编程评分。

In [11]:
from langchain.evaluation.qa import QAEvalChain

In [12]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(dataset, predictions, question_key="question", prediction_key="result")

我们可以将分级输出添加到“预测”字典中，然后计算分级的数量。

In [13]:
for i, prediction in enumerate(predictions):
    prediction['grade'] = graded_outputs[i]['text']

In [14]:
from collections import Counter
Counter([pred['grade'] for pred in predictions])

Counter({' CORRECT': 12, ' INCORRECT': 10})

我们还可以将错误示例的数据点过滤出来并查看它们。

In [15]:
incorrect = [pred for pred in predictions if pred['grade'] == " INCORRECT"]

In [16]:
incorrect[0]

{'question': 'What did the author write their dissertation on?',
 'answer': 'The author wrote their dissertation on applications of continuations.',
 'result': ' The author does not mention what their dissertation was on, so it is not known.',
 'grade': ' INCORRECT'}